# Групповой проект 2

Работу можно условно разделить на 2 этапа:


*   Подготовка датасета
*   Создание модели ML



# Подготовка датасета

In [ ]:
# Импортируем наши стандартные библиотеки
from copy import deepcopy

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

%matplotlib inline
sns.set()

In [ ]:
#полезные функции
def all_view(a):
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  print(a)
  pd.set_option('display.max_rows', 10)
  pd.set_option('display.max_columns', 20)

# Оцените точность модели
def acc(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)


Загрузим файл с исходным датсетом:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'
start_df = pd.read_csv(path + 'dataset_for_GP2.csv', index_col=0)
start_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ticker,publish_date,period,period until,eps,forecast_eps,income,forecast_income,Tax Effect Of Unusual Items,Tax Rate For Calcs,...,EBITDA,Net Income From Tax Loss Carryforward,Interest Paid Cfo,Securities Amortization,Interest Paid Cff,Duefrom Related Parties Non Current,Financial Assets Designatedas Fair Value Through Profitor Loss Total,Interest Received Cfi,open,close
N,,,,,,,,,,,,,,,,,,,,,
0,A,2022-08-16 00:00:00,2022-07,2022-07-31,"1,34","1,2","1,72B","1,64B",0,"0,1376",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"132,8899994","132,7700043"
1,A,2022-11-21 00:00:00,2022-10,2022-10-31,"1,53","1,38","1,85B","1,76B",0,"0,1412",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"145,6199951","145,1399994"
2,A,2023-02-28 00:00:00,2023-01,2023-01-31,"1,37","1,31","1,76B","1,7B",0,"0,1375",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"141,75","141,9700012"
3,A,2023-05-23 00:00:00,2023-04,2023-04-30,"1,27","1,26","1,72B","1,67B",0,"0,1375",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"129,0299988","128,6399994"
4,A,2023-08-15 00:00:00,2023-07,2023-07-31,"1,43","1,36","1,67B","1,66B",0,"0,1375",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"126,3099976","125,6500015"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,ZTS,2022-08-04 00:00:00,2022-06,2022-06-30,"1,2","1,21","2,05B","2,03B","-210447,7612","0,210448",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"181,2200012","174,3099976"
2556,ZTS,2022-11-03 00:00:00,2022-09,2022-09-30,"1,21","1,24",2B,2.083M,-3744000,"0,208",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"134,1499939","131,1399994"
2557,ZTS,2023-02-14 00:00:00,2022-12,2022-12-31,"1,15","1,15",2B,2B,"-445945,9459","0,222973",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"173,4400024","171,8999939"


Выведем основные параметры датасета:

In [ ]:
all_view(start_df.dtypes)

ticker                                                                   object
publish_date                                                             object
period                                                                   object
period until                                                             object
eps                                                                      object
forecast_eps                                                             object
income                                                                   object
forecast_income                                                          object
Tax Effect Of Unusual Items                                              object
Tax Rate For Calcs                                                       object
Normalized EBITDA                                                       float64
Total Unusual Items                                                     float64
Total Unusual Items Excluding Goodwill  

In [ ]:
start_df.isna().sum()

ticker                                                                     0
publish_date                                                               0
period                                                                     0
period until                                                               0
eps                                                                        0
                                                                        ... 
Duefrom Related Parties Non Current                                     2556
Financial Assets Designatedas Fair Value Through Profitor Loss Total    2558
Interest Received Cfi                                                   2557
open                                                                       0
close                                                                      0
Length: 328, dtype: int64

In [ ]:
unique_features = np.unique(start_df.columns)
print("Всего признаков:", len(unique_features))
print('Features', unique_features)

Всего признаков: 328
Features ['Accounts Payable' 'Accounts Receivable' 'Accrued Interest Receivable'
 'Accumulated Depreciation' 'Additional Paid In Capital'
 'Allowance For Doubtful Accounts Receivable' 'Amortization'
 'Amortization Cash Flow' 'Amortization Of Intangibles'
 'Amortization Of Intangibles Income Statement'
 'Amortization Of Securities' 'Asset Impairment Charge'
 'Assets Held For Sale Current' 'Available For Sale Securities'
 'Average Dilution Earnings' 'Basic Average Shares' 'Basic EPS'
 'Beginning Cash Position' 'Buildings And Improvements'
 'Capital Expenditure' 'Capital Expenditure Reported'
 'Capital Lease Obligations' 'Capital Stock' 'Cash And Cash Equivalents'
 'Cash Cash Equivalents And Short Term Investments' 'Cash Dividends Paid'
 'Cash Equivalents' 'Cash Financial'
 'Cash Flow From Continuing Financing Activities'
 'Cash Flow From Continuing Investing Activities'
 'Cash Flow From Continuing Operating Activities'
 'Cash Flow From Discontinued Operation'
 'Cash 

In [ ]:
unique_tickers = np.unique(start_df['ticker'])
print("Всего тикеров:", len(unique_tickers))
print('Tickers', unique_tickers)

Всего тикеров: 499
Tickers ['A' 'AAL' 'AAPL' 'ABBV' 'ABT' 'ACGL' 'ACN' 'ADBE' 'ADI' 'ADM' 'ADP'
 'ADSK' 'AEE' 'AEP' 'AES' 'AFL' 'AIG' 'AIZ' 'AJG' 'AKAM' 'ALB' 'ALGN'
 'ALK' 'ALL' 'ALLE' 'AMAT' 'AMCR' 'AMD' 'AME' 'AMGN' 'AMP' 'AMT' 'AMZN'
 'ANET' 'ANSS' 'AON' 'AOS' 'APA' 'APD' 'APH' 'APTV' 'ARE' 'ATO' 'ATVI'
 'AVB' 'AVGO' 'AVY' 'AWK' 'AXON' 'AXP' 'AZO' 'BA' 'BAC' 'BALL' 'BAX'
 'BBWI' 'BBY' 'BDX' 'BEN' 'BG' 'BIIB' 'BIO' 'BK' 'BKNG' 'BKR' 'BLK' 'BMY'
 'BR' 'BRO' 'BSX' 'BWA' 'BXP' 'C' 'CAG' 'CAH' 'CARR' 'CAT' 'CB' 'CBOE'
 'CBRE' 'CCI' 'CCL' 'CDAY' 'CDNS' 'CDW' 'CE' 'CEG' 'CF' 'CFG' 'CHD' 'CHRW'
 'CHTR' 'CI' 'CINF' 'CL' 'CLX' 'CMA' 'CMCSA' 'CME' 'CMG' 'CMI' 'CMS' 'CNC'
 'CNP' 'COF' 'COO' 'COP' 'COR' 'COST' 'CPB' 'CPRT' 'CPT' 'CRL' 'CRM'
 'CSCO' 'CSGP' 'CSX' 'CTAS' 'CTLT' 'CTRA' 'CTSH' 'CTVA' 'CVS' 'CVX' 'CZR'
 'D' 'DAL' 'DD' 'DE' 'DFS' 'DG' 'DGX' 'DHI' 'DHR' 'DIS' 'DLR' 'DLTR' 'DOV'
 'DOW' 'DPZ' 'DRI' 'DTE' 'DUK' 'DVA' 'DVN' 'DXC' 'DXCM' 'EA' 'EBAY' 'ECL'
 'ED' 'EFX' 'EG' 'EIX' 'EL' 'ELV' '

Начнем чистить данные, чтобы комфортно их использовать.
Рассмотрим типы данных для столбцов:

In [ ]:
df = start_df.copy()
df.select_dtypes(include=['object'])

,ticker,publish_date,period,period until,eps,forecast_eps,income,forecast_income,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized Income,Diluted EPS,Basic EPS,open,close
N,,,,,,,,,,,,,,,
0,A,2022-08-16 00:00:00,2022-07,2022-07-31,"1,34","1,2","1,72B","1,64B",0,"0,1376",329000000,"1,1","1,1","132,8899994","132,7700043"
1,A,2022-11-21 00:00:00,2022-10,2022-10-31,"1,53","1,38","1,85B","1,76B",0,"0,1412",368000000,"1,23","1,24","145,6199951","145,1399994"
2,A,2023-02-28 00:00:00,2023-01,2023-01-31,"1,37","1,31","1,76B","1,7B",0,"0,1375",352000000,"1,19","1,19","141,75","141,9700012"
3,A,2023-05-23 00:00:00,2023-04,2023-04-30,"1,27","1,26","1,72B","1,67B",0,"0,1375",302000000,"1,02","1,02","129,0299988","128,6399994"
4,A,2023-08-15 00:00:00,2023-07,2023-07-31,"1,43","1,36","1,67B","1,66B",0,"0,1375",111000000,"0,38","0,38","126,3099976","125,6500015"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,ZTS,2022-08-04 00:00:00,2022-06,2022-06-30,"1,2","1,21","2,05B","2,03B","-210447,7612","0,210448","529789552,2","1,12","1,13","181,2200012","174,3099976"
2556,ZTS,2022-11-03 00:00:00,2022-09,2022-09-30,"1,21","1,24",2B,2.083M,-3744000,"0,208",543256000,"1,13","1,13","134,1499939","131,1399994"
2557,ZTS,2023-02-14 00:00:00,2022-12,2022-12-31,"1,15","1,15",2B,2B,"-445945,9459","0,222973","462554054,1","0,99","0,99","173,4400024","171,8999939"


In [ ]:
wrong_col = ['eps', 'income']
df.drop(columns=wrong_col, inplace=True)
df

,ticker,publish_date,period,period until,forecast_eps,forecast_income,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,...,EBITDA,Net Income From Tax Loss Carryforward,Interest Paid Cfo,Securities Amortization,Interest Paid Cff,Duefrom Related Parties Non Current,Financial Assets Designatedas Fair Value Through Profitor Loss Total,Interest Received Cfi,open,close
N,,,,,,,,,,,,,,,,,,,,,
0,A,2022-08-16 00:00:00,2022-07,2022-07-31,"1,2","1,64B",0,"0,1376",496000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"132,8899994","132,7700043"
1,A,2022-11-21 00:00:00,2022-10,2022-10-31,"1,38","1,76B",0,"0,1412",551000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"145,6199951","145,1399994"
2,A,2023-02-28 00:00:00,2023-01,2023-01-31,"1,31","1,7B",0,"0,1375",502000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"141,75","141,9700012"
3,A,2023-05-23 00:00:00,2023-04,2023-04-30,"1,26","1,67B",0,"0,1375",472000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"129,0299988","128,6399994"
4,A,2023-08-15 00:00:00,2023-07,2023-07-31,"1,36","1,66B",0,"0,1375",227000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"126,3099976","125,6500015"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,ZTS,2022-08-04 00:00:00,2022-06,2022-06-30,"1,21","2,03B","-210447,7612","0,210448",761000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"181,2200012","174,3099976"
2556,ZTS,2022-11-03 00:00:00,2022-09,2022-09-30,"1,24",2.083M,-3744000,"0,208",853000000.0,-18000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"134,1499939","131,1399994"
2557,ZTS,2023-02-14 00:00:00,2022-12,2022-12-31,"1,15",2B,"-445945,9459","0,222973",691000000.0,-2000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"173,4400024","171,8999939"


Видим, что есть много столбцов, с типом object, которое по логике должны иметь тип float. Поэтому, преобразуем их в числове данные, B и M преобразуем в миллиарды и миллионы соответственно.  Также, необходимо заменить символ "--" на NaN.

In [ ]:
def prepare_dataset(df):
    columns_for_astype = ['Tax Rate For Calcs', 'Normalized Income', 'Diluted EPS', 'open', 'close', 'Basic EPS', 'Tax Effect Of Unusual Items']
    df[columns_for_astype] = df[columns_for_astype].replace(',', '.', regex=True).replace('--', np.nan, regex=True).astype(float)
    columns_for_change_letters = ['forecast_income', 'forecast_eps']
    df[columns_for_change_letters] = df[columns_for_change_letters].replace('M', '1000000', regex=True).replace('B', '1000000000', regex=True).replace(',', '.', regex=True).replace('--', np.nan, regex=True).astype(float)
    df.select_dtypes(include=['object'])
    wrong_col = ['eps', 'income']
    df.drop(columns=wrong_col, inplace=True)
    return df


# Работа с пропусками
Как видно, все столбцы типа object теперь обоснованы.
Рассмотрим пропуски:

In [ ]:
msno.bar(df)

In [ ]:
msno.matrix(df)

In [ ]:
df_nulls = df.iloc[:, [i for i, n in enumerate(np.var(df.isnull(), axis = 'rows')) if n > 0]]


In [ ]:
df_nulls.isnull().corr()

,forecast_eps,forecast_income,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Reconciled Depreciation,Reconciled Cost Of Revenue,EBIT,Net Interest Income,Interest Expense,...,Cash Flow From Discontinued Operation,Net Income Extraordinary,EBITDA,Net Income From Tax Loss Carryforward,Interest Paid Cfo,Securities Amortization,Interest Paid Cff,Duefrom Related Parties Non Current,Financial Assets Designatedas Fair Value Through Profitor Loss Total,Interest Received Cfi
forecast_eps,1.000000,1.000000,0.136908,0.055634,0.055634,0.301357,0.129636,-0.010442,-0.005576,-0.012813,...,0.003618,0.001516,0.010564,0.001237,0.001958,0.001958,0.001237,0.001516,0.000875,0.001237
forecast_income,1.000000,1.000000,0.136908,0.055634,0.055634,0.301357,0.129636,-0.010442,-0.005576,-0.012813,...,0.003618,0.001516,0.010564,0.001237,0.001958,0.001958,0.001237,0.001516,0.000875,0.001237
Normalized EBITDA,0.136908,0.136908,1.000000,0.143477,0.143477,0.287054,0.946883,0.730224,0.013105,-0.068599,...,0.009990,0.011072,0.077159,0.009038,0.014299,0.014299,0.009038,0.011072,0.006390,0.009038
Total Unusual Items,0.055634,0.055634,0.143477,1.000000,1.000000,0.102077,0.118728,0.095734,0.006310,0.056712,...,-0.018753,0.019950,0.011716,0.016286,0.025765,0.025765,0.016286,0.019950,0.011513,0.016286
Total Unusual Items Excluding Goodwill,0.055634,0.055634,0.143477,1.000000,1.000000,0.102077,0.118728,0.095734,0.006310,0.056712,...,-0.018753,0.019950,0.011716,0.016286,0.025765,0.025765,0.016286,0.019950,0.011513,0.016286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Securities Amortization,0.001958,0.001958,0.014299,0.025765,0.025765,0.006496,0.015102,0.010442,0.005576,-0.152788,...,-0.003618,-0.001516,-0.010564,-0.001237,-0.001958,1.000000,-0.001237,-0.001516,-0.000875,-0.001237
Interest Paid Cff,0.001237,0.001237,0.009038,0.016286,0.016286,0.004106,0.009545,0.006600,0.003524,0.008099,...,-0.002287,-0.000958,-0.006677,-0.000782,-0.001237,-0.001237,1.000000,-0.000958,-0.000553,-0.000782
Duefrom Related Parties Non Current,0.001516,0.001516,0.011072,0.019950,0.019950,0.005030,0.011693,0.008085,0.004317,0.009921,...,-0.002802,-0.001174,-0.008179,-0.000958,-0.001516,-0.001516,-0.000958,1.000000,-0.000677,-0.000958
Financial Assets Designatedas Fair Value Through Profitor Loss Total,0.000875,0.000875,0.006390,0.011513,0.011513,0.002903,0.006748,0.004666,-0.156904,-0.068276,...,-0.001617,-0.000677,-0.004721,-0.000553,-0.000875,-0.000875,-0.000553,-0.000677,1.000000,-0.000553


# Радикальный метод борьбы с пропусками
Удалим все признаки, в которых есть хотя бы 1 пропуск

In [ ]:
df_clear = df.dropna(axis=1)

In [ ]:
df_clear

,ticker,publish_date,period,period until,Tax Effect Of Unusual Items,Tax Rate For Calcs,Net Income From Continuing Operation Net Minority Interest,Normalized Income,Net Income From Continuing And Discontinued Operation,Diluted Average Shares,...,Free Cash Flow,End Cash Position,Beginning Cash Position,Financing Cash Flow,Cash Flow From Continuing Financing Activities,Investing Cash Flow,Cash Flow From Continuing Investing Activities,Operating Cash Flow,open,close
N,,,,,,,,,,,,,,,,,,,,,
0,A,2022-08-16 00:00:00,2022-07,2022-07-31,0.000000e+00,0.137600,329000000,329000000.0,329000000,299000000,...,244000000,1073000000,1188000000,-362000000,-362000000,-73000000,-73000000,326000000,132.889999,132.770004
1,A,2022-11-21 00:00:00,2022-10,2022-10-31,0.000000e+00,0.141200,368000000,368000000.0,368000000,297000000,...,378000000,1056000000,1073000000,-341000000,-341000000,-110000000,-110000000,448000000,145.619995,145.139999
2,A,2023-02-28 00:00:00,2023-01,2023-01-31,0.000000e+00,0.137500,352000000,352000000.0,352000000,297000000,...,220000000,1253000000,1056000000,-17000000,-17000000,-104000000,-104000000,296000000,141.750000,141.970001
3,A,2023-05-23 00:00:00,2023-04,2023-04-30,0.000000e+00,0.137500,302000000,302000000.0,302000000,297000000,...,341000000,1178000000,1253000000,-390000000,-390000000,-77000000,-77000000,398000000,129.029999,128.639999
4,A,2023-08-15 00:00:00,2023-07,2023-07-31,0.000000e+00,0.137500,111000000,111000000.0,111000000,295000000,...,481000000,1332000000,1178000000,-322000000,-322000000,-89000000,-89000000,562000000,126.309998,125.650002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,ZTS,2022-08-04 00:00:00,2022-06,2022-06-30,-2.104478e+05,0.210448,529000000,529789552.2,529000000,471500000,...,190000000,2652000000,3135000000,-610000000,-610000000,-203000000,-203000000,336000000,181.220001,174.309998
2556,ZTS,2022-11-03 00:00:00,2022-09,2022-09-30,-3.744000e+06,0.208000,529000000,543256000.0,529000000,469100000,...,372000000,2507000000,2652000000,-529000000,-529000000,-124000000,-124000000,526000000,134.149994,131.139999
2557,ZTS,2023-02-14 00:00:00,2022-12,2022-12-31,-4.459459e+05,0.222973,461000000,462554054.1,461000000,466740000,...,570000000,3581000000,2507000000,780000000,780000000,-438000000,-438000000,741000000,173.440002,171.899994


In [ ]:
len(df.columns)

326

In [ ]:
len(df_nulls.columns)

284

In [ ]:
number_of_nans = df.isna().sum() # количество пропусков в столбце
all_view(number_of_nans)

Убираем столбцы, в которых не более 100 значений.

In [ ]:
min_data = 100 #минимальное количество заданыых значений в одном признаке
for i in number_of_nans.index:
    if number_of_nans[i] > len(df) - min_data:
        df = df.drop(columns=i)
len(df.columns)

283

Надо убрать строки, в котором хотя бы половина значений это пропуски:

In [ ]:
threshold = len(df.columns) // 2  # Половина числа столбцов

df = df.dropna(thresh=threshold)


In [ ]:
len(df)

2221

# Первый способ работы с датами
Разделим даты на отдельные признаки, как число, месяц и год (сразу запишем в виде функций, чтобы пробовать разные комбинации):

In [ ]:
def publish_date_mode_1(df):
    df[['publish_year', 'publish_months', 'publish_day' ]] = df['publish_date'].str.split('-', expand=True)
    # Преобразуем новые столбцы в числовой формат
    df['publish_year'] = df['publish_year'].astype(int)
    df['publish_months'] = df['publish_months'].astype(int)
    df['publish_day'] = df['publish_day'].replace('00:00:00','', regex=True).astype(int)
    df.drop(columns=['publish_date'], inplace=True)
    return df

In [ ]:
def period_mode_1(df):
    df[['period_year', 'period_months']] = df['period'].str.split('-', expand=True)
    # Преобразуем новые столбцы в числовой формат
    df['period_year'] = df['period_year'].astype(int)
    df['period_months'] = df['period_months'].astype(int)
    df.drop(columns=['period'], inplace=True)
    return df

In [ ]:
def until_date_mode_1(df):
    df[['until_year', 'until_months', 'until_day' ]] = df['period until'].str.split('-', expand=True)
    # Преобразуем новые столбцы в числовой формат
    df['until_year'] = df['until_year'].astype(int)
    df['until_months'] = df['until_months'].astype(int)
    df['until_day'] = df['until_day'].replace('00:00:00','', regex=True).astype(int)
    df.drop(columns=['period until'], inplace=True)
    return df

# Второй способ работы с датами
Попробуем другой вариант работы с датами, а именно, оставим только период, но преобразуем его в формат год + сезон, которые будут считаться категориальными признаками

In [ ]:
def only_year_and_season(df):
    df.drop(columns=['publish_date'], inplace=True)
    df.drop(columns=['period until'], inplace=True)

    df[['period_year', 'period_months']] = df['period'].str.split('-', expand=True)
    df.drop(columns=['period'], inplace=True)
    seasons_dict = {'01': 'winter',
                    '02': 'winter',
                    '03': 'spring',
                    '04': 'spring',
                    '05': 'spring',
                    '06': 'summer',
                    '07': 'summer',
                    '08': 'summer',
                    '09': 'autumn',
                    '10': 'autumn',
                    '11': 'autumn',
                    '12': 'winter'
                }
    df['Seasons'] = df['period_months'].apply(lambda x: seasons_dict[x])
    df['period_year'] = df['period_year'].astype(int)
    df.drop(columns=['period_months'], inplace=True)
    return df


In [ ]:
df = pd.get_dummies(df, columns=['ticker'])
df = pd.get_dummies(df, columns=['Seasons'])

In [ ]:
def dummy(df, parametr):
    if parametr == 1:
        df = pd.get_dummies(df, columns=['ticker'])
        df = pd.get_dummies(df, columns=['Seasons'])
    else:
        df = pd.get_dummies(df, columns=['ticker'])
    return df

In [ ]:
df_clear = only_year_and_season(df_clear)

<ipython-input-13-2fc5f2ba4902>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['publish_date'], inplace=True)
<ipython-input-13-2fc5f2ba4902>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['period until'], inplace=True)
<ipython-input-13-2fc5f2ba4902>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['period_year', 'period_months']] = df['

In [ ]:
df_clear = pd.get_dummies(df_clear, columns=['ticker'])
df_clear = pd.get_dummies(df_clear, columns=['Seasons'])

Рассмотрим корреляции между параметрами:

In [ ]:

# Рассчитайте матрицу корреляций
correlation_matrix = df.corr()

# Инициализируйте список для хранения столбцов, которые нужно удалить
columns_to_drop = []

# Определите столбцы для удаления на основе условий
for i in range(len(correlation_matrix.columns)-1):
    for j in range(i+1, len(correlation_matrix.columns)-1):
        if i ==j:
          continue
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            # Если коэффициент корреляции больше 0.9
            if df[correlation_matrix.columns[i]].isna().sum() > df[correlation_matrix.columns[j]].isna().sum():
                columns_to_drop.append(correlation_matrix.columns[j])
            elif df[correlation_matrix.columns[i]].isna().sum() < df[correlation_matrix.columns[j]].isna().sum():
                columns_to_drop.append(correlation_matrix.columns[i])
            else:
                # Если количество пропусков равно, удалите последний столбец
                columns_to_drop.append(correlation_matrix.columns[j])

columns_to_drop = np.unique(columns_to_drop)
columns_to_drop = columns_to_drop[columns_to_drop != 'close']
# Удалите выбранные столбцы
df = df.drop(columns=columns_to_drop)

<ipython-input-100-1f71209a7da1>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


# Заполнение пропусков

In [ ]:
#удалить по пороговому количеству
def noninf_col_del(df1, threshold):
    number_of_nans = df1.isna().sum(axis=0)
    for i in number_of_nans.index:
        if number_of_nans[i] > len(df1) - threshold:
            df1 = df1.drop(columns=i)
    return df1


#удалить все признаки с пропусками
def get_clear(df):
    df = df.dropna(axis=1)
    return df

#заполнить средним
def get_mean(df):
    df.fillna(df.mean(), inplace = True)
    return df

#заполнить медианой
def get_median(df):
    df.fillna(df.median(), inplace = True)
    return df

#удалить строки по граничному значению
def string_del(df, threshold):
    df = df.dropna(thresh=threshold)
    return df

# Создание модели ML

Дополнительные библиотеки

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

Для того, чтобы создать модель машинного обучения необходимо задать таргет и разделить датасет на признаки и таргет. Поскольку мы решаем задачу классификации, то надо создать столбец target, который будет определен как отношение open и close, округленное с точность до целых, т.е. 0 - цена упала после отчета, 1 - выросла.

In [ ]:
def create_target(df):
    df['target'] = (df['close']/df['open']).astype(int)
    df.drop(columns=['close'], inplace=True)
    return df

Разделим датасет на таргет и прзнаки:

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

In [ ]:
# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создайте и настройте классификатор HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=50, random_state=42)
clf.fit(X_train, y_train)

# Сделайте предсказания на тестовых данных
y_pred = clf.predict(X_test)

In [ ]:
acc(y_test, y_pred)

Accuracy: 0.5235955056179775


In [ ]:
#df_clear['target'] = (df_clear['close']/df_clear['open']).astype(int)
#df_clear.drop(columns=['close'], inplace=True)
X_clear = df_clear.drop(columns=['target'])
y_clear = df_clear['target']

# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_clear, y_clear, test_size=0.2, random_state=42)

# Создайте и настройте классификатор HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=50, random_state=42)
clf.fit(X_train, y_train)

# Сделайте предсказания на тестовых данных
y_pred = clf.predict(X_test)
acc(y_test, y_pred)

Accuracy: 0.525390625


In [ ]:
# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_clear, y_clear, test_size=0.2)

# Создайте экземпляр классификатора случайного леса
rf_classifier = RandomForestClassifier(n_estimators=100)

# Обучите модель на обучающих данных
rf_classifier.fit(X_train, y_train)

# Сделайте предсказания на тестовых данных
y_pred = rf_classifier.predict(X_test)

acc(y_test=y_test, y_pred=y_pred)







Accuracy: 0.54296875


#Функция HGBC

In [ ]:
def create_HGBC(df, estimators, parametrs):


    X = df.drop(columns=['target'])
    y = df['target']
    # Разделите данные на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Нормализуйте признаки
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Создайте и настройте классификатор HistGradientBoostingClassifier
    clf = HistGradientBoostingClassifier(max_iter=50, random_state=42)
    clf.fit(X_train_scaled, y_train)

    # Сделайте предсказания на тестовых данных
    y_pred = clf.predict(X_test_scaled)
    parametrs = np.append(parametrs, accuracy_score(y_test, y_pred))
    return parametrs


array([5.        , 6.        , 0.52539062])

#Функция RF

In [ ]:


def create_rf(df, parametrs):
    # Разделите данные на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X_clear, y_clear, test_size=0.2)

    # Определите сетку параметров для GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],  # Разные значения числа деревьев
        'max_depth': [None, 10, 20],  # Разные значения максимальной глубины деревьев
        # Другие параметры случайного леса, которые вы хотите настроить
    }

    # Создайте экземпляр классификатора случайного леса
    rf_classifier = RandomForestClassifier()

    # Создайте объект GridSearchCV
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3)

    # Обучите модель на обучающих данных с использованием GridSearchCV
    grid_search.fit(X_train, y_train)

    # Получите наилучшие параметры из GridSearchCV
    best_params = grid_search.best_params_
    print("Наилучшие параметры:", best_params)

    # Используйте модель с наилучшими параметрами для предсказания
    best_rf_classifier = grid_search.best_estimator_
    y_pred = best_rf_classifier.predict(X_test)

    # Оцените точность модели
    accuracy = accuracy_score(y_test, y_pred)
    print("Точность модели:", accuracy)

    # Добавьте результаты в параметры
    parametrs = np.append(parametrs, accuracy)
    return parametrs


# Фунуция GBC

In [ ]:
def create_GBC(df,  parametrs, n_estimators=100,  learning_rate=0.1, max_depth=3):
    # Выделение признаков и целевой переменной
    X = df.drop(columns=['target'])
    y = df['target']

    # Разделение данных на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Нормализация данных
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Создание экземпляра Gradient Boosting Classifier
    gbc = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)

    # Оптимизация параметров с использованием GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5]
    }

    grid_search = GridSearchCV(estimator=gbc, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train_scaled, y_train)

    # Получение наилучших параметров
    best_params = grid_search.best_params_
    print("Наилучшие параметры:", best_params)

    # Обучение модели с наилучшими параметрами
    optimized_gbc = GradientBoostingClassifier(n_estimators=best_params['n_estimators'],
                                               learning_rate=best_params['learning_rate'],
                                               max_depth=best_params['max_depth'],
                                               random_state=42)
    optimized_gbc.fit(X_train_scaled, y_train)

    # Предсказание на тестовых данных
    y_pred = optimized_gbc.predict(X_test_scaled)

    # Вычисление точности модели
    accuracy = accuracy_score(y_test, y_pred)
    print("Точность модели:", accuracy)
    # Добавьте результаты в параметры
    parametrs = np.append(parametrs, accuracy)
    return parametrs

In [ ]:


def create_CatBoost(df, parametrs, n_estimators=100, learning_rate=0.1, max_depth=6):
    # Выделение признаков и целевой переменной
    X = df.drop(columns=['target'])
    y = df['target']

    # Разделение данных на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Создание экземпляра CatBoost Classifier
    catboost = CatBoostClassifier(iterations=n_estimators, learning_rate=learning_rate, depth=max_depth, verbose=0)

    # Оптимизация параметров с использованием GridSearchCV
    param_grid = {
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'depth': [3, 4, 5]
    }

    grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train, verbose=False, plot=False)

    # Получение наилучших параметров
    best_params = grid_search.best_params_
    print("Наилучшие параметры:", best_params)

    # Обучение модели с наилучшими параметрами
    optimized_catboost = CatBoostClassifier(iterations=best_params['iterations'],
                                             learning_rate=best_params['learning_rate'],
                                             depth=best_params['depth'],
                                             verbose=0)
    optimized_catboost.fit(X_train, y_train, verbose=False)

    # Предсказание на тестовых данных
    y_pred = optimized_catboost.predict(X_test)

    # Вычисление точности модели
    accuracy = accuracy_score(y_test, y_pred)
    print("Точность модели:", accuracy)

    # Добавьте результаты в параметры
    parametrs = np.append(parametrs, accuracy)
    return parametrs


#Финальный цикл

,ticker,publish_date,period,period until,eps,forecast_eps,income,forecast_income,Tax Effect Of Unusual Items,Tax Rate For Calcs,...,EBITDA,Net Income From Tax Loss Carryforward,Interest Paid Cfo,Securities Amortization,Interest Paid Cff,Duefrom Related Parties Non Current,Financial Assets Designatedas Fair Value Through Profitor Loss Total,Interest Received Cfi,open,close
N,,,,,,,,,,,,,,,,,,,,,
0,A,2022-08-16 00:00:00,2022-07,2022-07-31,"1,34","1,2","1,72B","1,64B",0,"0,1376",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"132,8899994","132,7700043"
1,A,2022-11-21 00:00:00,2022-10,2022-10-31,"1,53","1,38","1,85B","1,76B",0,"0,1412",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"145,6199951","145,1399994"
2,A,2023-02-28 00:00:00,2023-01,2023-01-31,"1,37","1,31","1,76B","1,7B",0,"0,1375",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"141,75","141,9700012"
3,A,2023-05-23 00:00:00,2023-04,2023-04-30,"1,27","1,26","1,72B","1,67B",0,"0,1375",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"129,0299988","128,6399994"
4,A,2023-08-15 00:00:00,2023-07,2023-07-31,"1,43","1,36","1,67B","1,66B",0,"0,1375",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"126,3099976","125,6500015"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,ZTS,2022-08-04 00:00:00,2022-06,2022-06-30,"1,2","1,21","2,05B","2,03B","-210447,7612","0,210448",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"181,2200012","174,3099976"
2556,ZTS,2022-11-03 00:00:00,2022-09,2022-09-30,"1,21","1,24",2B,2.083M,-3744000,"0,208",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"134,1499939","131,1399994"
2557,ZTS,2023-02-14 00:00:00,2022-12,2022-12-31,"1,15","1,15",2B,2B,"-445945,9459","0,222973",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"173,4400024","171,8999939"


In [ ]:
result_df

,Граница информативного столбца,Метод обработки даты,Чистим ли короткие строки,Граница для информативной строки,Метод заполнения пропусков:,Метод ML,Accuracy
0,0.0,0.0,0.0,0.0,0.0,HGBC,0.576171875
1,0.0,0.0,0.0,0.0,0.0,RF,0.56640625
2,0.0,0.0,0.0,0.0,1.0,HGBC,0.546875
3,0.0,0.0,0.0,0.0,1.0,RF,0.515625
4,0.0,0.0,0.0,0.0,2.0,HGBC,0.5625
...,...,...,...,...,...,...,...
139,500.0,1.0,1.0,119.0,0.0,RF,0.51171875
140,500.0,1.0,1.0,119.0,1.0,HGBC,0.54296875
141,500.0,1.0,1.0,119.0,1.0,RF,0.560546875
142,500.0,1.0,1.0,119.0,2.0,HGBC,0.564453125


In [ ]:
import pdb




In [ ]:
param

array([  0.,   1.,   0., 274.,   2.,  50.])

In [ ]:
param = np.array([])
cntr = 0
models = ['HGBC', 'RF']
columns = ['Граница информативного столбца', 'Метод обработки даты', 'Чистим ли короткие строки ', 'Граница для информативной строки', 'Метод заполнения пропусков:', 'Метод ML', 'Accuracy']
result_df = pd.DataFrame(columns=columns)
df_ = start_df.copy()
df_ = create_target(prepare_dataset(df_))

for i in range(0, 501, 100): #цикл по 1
    df_1 = df_.copy()
    df_1 = noninf_col_del(df_1, threshold=i)
    if len(param) < 5:
        param = np.append(param, i) #1
    else:
        param[0] = i
    for j in range(2): #цикл по 2
        df_2 = df_1.copy()
        if len(param) < 5:
            if j == 0:
                df_2 = publish_date_mode_1(df_2)
                df_2 = perod_mode_1(df_2)
                df_2 = until_date_mode_1(df_2)
                param = np.append(param, j) #2
            else:
                df_2 = only_year_and_season(df_2)
                param = np.append(param, j) #2
        if len(param) == 5:
            if j == 0:
                df_2 = publish_date_mode_1(df_2)
                df_2 = perod_mode_1(df_2)
                df_2 = until_date_mode_1(df_2)
                param[1] = j #2
            else:
                df_2 = only_year_and_season(df_2)
                param[1] = j #2

        df_2 = dummy(df_2, j) #final_dataset
        for str_clean_mode in range(2):
            df_3 = df_2.copy()
            if len(param) < 5:
                if str_clean_mode == 0:
                    param = np.append(param, 0) #3
                    param = np.append(param, 0) #4
                else:
                    for m in range(len(df_3.columns)//6, len(df_3.columns)//3, len(df_3.columns)//6):
                        df_3 = string_del(df_3, l)
                        param = np.append(param, 1) #3
                        param = np.append(param, m) #4
            if len(param) >= 5:
                if str_clean_mode == 0:
                    param[2] = 0 #3
                    param[3] = 0 #4
                else:
                    for m in range(len(df_3.columns)//6, len(df_3.columns)//3, len(df_3.columns)//6):
                        df_3 = string_del(df_3, l)
                        param[2] = 1 #3
                        param[3] = m #4
            for nans_mode in range(3):
                df_4 = df_3.copy()
                if len(param) < 5:
                    if nans_mode == 0:
                        df_4 = get_clear(df_4)
                        param = np.append(param, nans_mode) #5
                    elif nans_mode == 1:
                        df_4 = get_mean(df_4)
                        param = np.append(param, nans_mode) #5
                    elif nans_mode == 2:
                        df_4 = get_median(df_4)
                        param = np.append(param, nans_mode) #5
                if len(param) == 5:
                    if nans_mode == 0:
                        df_4 = get_clear(df_4)
                        param[4] = nans_mode #5
                    elif nans_mode == 1:
                        df_4 = get_mean(df_4)
                        param[4] = nans_mode #5
                    elif nans_mode == 2:
                        df_4 = get_median(df_4)
                        param[4] = nans_mode #5

                for method in models:
                    if method == 'HGBC':
                        param_last = param.copy()
                        df_last = df_4.copy()
                        param_last = np.append(param_last, method) #6
                        param_last = create_HGBC(df_last, estimators=100, parametrs=param_last) #7
                        result_df.loc[len(result_df)] = param_last
                        print(f"param_last={param_last}")
                    elif method == 'RF':
                        param_last = param.copy()
                        df_last = df_4.copy()
                        param_last = np.append(param_last, method) #6
                        param_last = create_rf(df_last,  parametrs=param_last) #7
                        print(f"param_last={param_last}")
                        result_df.loc[len(result_df)] = param_last
              #      elif method == 'GBC':
            #            param_last = param.copy()
          #              df_last = df_4.copy()
          #              param_last = np.append(param_last, method) #6
         #               param_last = create_GBC(df_last,  parametrs=param_last) #7
          #              result_df.loc[len(result_df)] = param_last


param_last=['0.0' '0.0' '0.0' '0.0' '0.0' 'HGBC' '0.576171875']
Наилучшие параметры: {'max_depth': None, 'n_estimators': 300}
Точность модели: 0.56640625
param_last=['0.0' '0.0' '0.0' '0.0' '0.0' 'RF' '0.56640625']
param_last=['0.0' '0.0' '0.0' '0.0' '1.0' 'HGBC' '0.546875']
Наилучшие параметры: {'max_depth': None, 'n_estimators': 300}
Точность модели: 0.515625
param_last=['0.0' '0.0' '0.0' '0.0' '1.0' 'RF' '0.515625']
param_last=['0.0' '0.0' '0.0' '0.0' '2.0' 'HGBC' '0.5625']
Наилучшие параметры: {'max_depth': None, 'n_estimators': 300}
Точность модели: 0.583984375
param_last=['0.0' '0.0' '0.0' '0.0' '2.0' 'RF' '0.583984375']
param_last=['0.0' '0.0' '1.0' '138.0' '0.0' 'HGBC' '0.576171875']
Наилучшие параметры: {'max_depth': None, 'n_estimators': 100}
Точность модели: 0.556640625
param_last=['0.0' '0.0' '1.0' '138.0' '0.0' 'RF' '0.556640625']
param_last=['0.0' '0.0' '1.0' '138.0' '1.0' 'HGBC' '0.546875']
Наилучшие параметры: {'max_depth': 20, 'n_estimators': 300}
Точность модели: 0.53

In [ ]:
result_df.to_csv('result.csv', index = False)
#files.download('/content/result.csv')

# Работа с результатами

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'
res_df = pd.read_csv(path + 'result.csv')
res_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Граница информативного столбца,Метод обработки даты,Чистим ли короткие строки,Граница для информативной строки,Метод заполнения пропусков:,Метод ML,Accuracy
0,0.0,0.0,0.0,0.0,0.0,HGBC,0.576172
1,0.0,0.0,0.0,0.0,0.0,RF,0.566406
2,0.0,0.0,0.0,0.0,1.0,HGBC,0.546875
3,0.0,0.0,0.0,0.0,1.0,RF,0.515625
4,0.0,0.0,0.0,0.0,2.0,HGBC,0.562500
...,...,...,...,...,...,...,...
139,500.0,1.0,1.0,119.0,0.0,RF,0.511719
140,500.0,1.0,1.0,119.0,1.0,HGBC,0.542969
141,500.0,1.0,1.0,119.0,1.0,RF,0.560547
142,500.0,1.0,1.0,119.0,2.0,HGBC,0.564453


In [ ]:
clear_df = res_df.drop_duplicates(subset=['Accuracy'])
top_10_rows = clear_df.nlargest(10, 'Accuracy')
top_10_rows
top_10_rows.columns = ['Минимальное количество значений в столбце', 'Метод обработки даты', 'Убирались ли строки с малым кол-вом признаков', 'Минимальное количество признаков', 'Метод заполнения пропусков', 'Метод ML', 'Accuracy']
top_10_rows['Убирались ли строки с малым кол-вом признаков'].replace({0: 'No', 1: 'Yes'}, inplace=True)
top_10_rows['Минимальное количество значений в столбце'] = top_10_rows['Минимальное количество значений в столбце'].astype(int)
top_10_rows['Минимальное количество признаков'] = top_10_rows['Минимальное количество признаков'].astype(int)
top_10_rows['Метод обработки даты'] = top_10_rows['Метод обработки даты'].astype(int)
top_10_rows['Метод заполнения пропусков'] = top_10_rows['Метод заполнения пропусков'].astype(int)
top_10_rows

,Минимальное количество значений в столбце,Метод обработки даты,Убирались ли строки с малым кол-вом признаков,Минимальное количество признаков,Метод заполнения пропусков,Метод ML,Accuracy
28,100,0,No,0,2,HGBC,0.585938
5,0,0,No,0,2,RF,0.583984
14,0,1,No,0,1,HGBC,0.580078
0,0,0,No,0,0,HGBC,0.576172
40,100,1,No,0,2,HGBC,0.572266
50,200,0,No,0,1,HGBC,0.570312
52,200,0,No,0,2,HGBC,0.568359
1,0,0,No,0,0,RF,0.566406
136,500,1,No,0,2,HGBC,0.564453
4,0,0,No,0,2,HGBC,0.562500



Параметры на выходе:

1.   Минимальное количесвто значений в столбце
2.   Метод обработки даты:
        0 - разделение дат на год-число-месяц и преобразование в int
        1- оставляем только период и преобразуем в 2 признака - год и время года
3.   Убирались ли строки с малым числом признаков
3.   Минимальное количество признаков
4.   Метод заполнения пропусков:
     0 - удаление всех признаков с пропусками
     1 - заполнение средним арифметическим
     2 - заполнение медианой

7.   Метод ML:
        HGBC
        RFC
8.   Accuracy



